# Introduzione a Probabilità e Statistica
In questo notebook, esploreremo alcuni dei concetti di cui abbiamo parlato in precedenza. Molti concetti di probabilità e statistica sono ben rappresentati nelle principali librerie per l'elaborazione dei dati in Python, come `numpy` e `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Variabili Casuali e Distribuzioni
Iniziamo estraendo un campione di 30 valori da una distribuzione uniforme da 0 a 9. Calcoleremo anche la media e la varianza.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Per stimare visivamente quante diverse valori ci sono nel campione, possiamo tracciare l'**istogramma**:


In [ ]:
plt.hist(sample)
plt.show()

## Analisi dei dati reali

Media e varianza sono molto importanti quando si analizzano dati del mondo reale. Carichiamo i dati sui giocatori di baseball da [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights)


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Stiamo usando un pacchetto chiamato [**Pandas**](https://pandas.pydata.org/) per l'analisi dei dati. Parleremo più nel dettaglio di Pandas e di come lavorare con i dati in Python più avanti in questo corso.

Calcoliamo i valori medi per età, altezza e peso:


In [ ]:
df[['Age','Height','Weight']].mean()

Ora concentriamoci sull'altezza e calcoliamo la deviazione standard e la varianza:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Oltre alla media, è sensato esaminare il valore mediano e i quartili. Possono essere visualizzati usando un **box plot**:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Possiamo anche creare diagrammi a scatola di sottoinsiemi del nostro dataset, ad esempio, raggruppati per ruolo del giocatore.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Nota**: Questo diagramma suggerisce che, in media, l'altezza dei primi base è maggiore dell'altezza dei secondi base. Più avanti impareremo come possiamo testare questa ipotesi in modo più formale e come dimostrare che i nostri dati sono statisticamente significativi per mostrarlo.  

Età, altezza e peso sono tutte variabili casuali continue. Cosa pensi sia la loro distribuzione? Un buon modo per scoprirlo è tracciare l'istogramma dei valori: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Distribuzione Normale

Creiamo un campione artificiale di pesi che segue una distribuzione normale con la stessa media e varianza dei nostri dati reali:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Poiché la maggior parte dei valori nella vita reale segue una distribuzione normale, non dovremmo utilizzare un generatore di numeri casuali uniforme per generare dati campione. Ecco cosa succede se proviamo a generare pesi con una distribuzione uniforme (generata da `np.random.rand`):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Intervalli di confidenza

Calcoliamo ora gli intervalli di confidenza per i pesi e le altezze dei giocatori di baseball. Useremo il codice [da questa discussione su stackoverflow](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Test di Ipotesi

Esploriamo diversi ruoli nel nostro dataset di giocatori di baseball:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Mettiamo alla prova l'ipotesi che i prima base siano più alti dei seconda base. Il modo più semplice per farlo è testare gli intervalli di confidenza:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Possiamo vedere che gli intervalli non si sovrappongono.

Un modo statisticamente più corretto per dimostrare l'ipotesi è utilizzare un **test t di Student**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

I due valori restituiti dalla funzione `ttest_ind` sono:
* il p-value può essere considerato come la probabilità che due distribuzioni abbiano la stessa media. Nel nostro caso, è molto basso, il che significa che ci sono forti prove a supporto che i primi basi siano più alti.
* il t-value è il valore intermedio della differenza media normalizzata che viene utilizzato nel test t, ed è confrontato con un valore soglia per un dato livello di confidenza.


## Simulazione di una Distribuzione Normale con il Teorema del Limite Centrale

Il generatore pseudo-casuale in Python è progettato per fornirci una distribuzione uniforme. Se vogliamo creare un generatore per una distribuzione normale, possiamo usare il teorema del limite centrale. Per ottenere un valore distribuito normalmente, calcoleremo semplicemente la media di un campione generato uniformemente.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Correlazione e Evil Baseball Corp

La correlazione ci permette di trovare relazioni tra sequenze di dati. Nel nostro esempio giocattolo, immaginiamo che ci sia una cattiva corporazione di baseball che paga i suoi giocatori in base alla loro altezza - più il giocatore è alto, più soldi riceve. Supponiamo che ci sia uno stipendio base di $1000 e un bonus aggiuntivo da $0 a $100, a seconda dell'altezza. Prenderemo i giocatori reali della MLB e calcoleremo i loro stipendi immaginari:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Calcoliamo ora la covarianza e la correlazione di quelle sequenze. `np.cov` ci fornirà una cosiddetta **matrice di covarianza**, che è un'estensione della covarianza a variabili multiple. L'elemento $M_{ij}$ della matrice di covarianza $M$ è una correlazione tra le variabili di input $X_i$ e $X_j$, e i valori diagonali $M_{ii}$ sono la varianza di $X_{i}$. Analogamente, `np.corrcoef` ci fornirà la **matrice di correlazione**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Una correlazione pari a 1 significa che esiste una forte **relazione lineare** tra due variabili. Possiamo vedere visivamente la relazione lineare tracciando un valore in funzione dell'altro:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Vediamo cosa succede se la relazione non è lineare. Supponiamo che la nostra azienda abbia deciso di nascondere l'ovvia dipendenza lineare tra altezze e salari, e abbia introdotto una certa non linearità nella formula, come `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In questo caso, la correlazione è leggermente inferiore, ma è ancora piuttosto alta. Ora, per rendere la relazione ancora meno evidente, potremmo voler aggiungere un po' di casualità extra aggiungendo una variabile casuale allo stipendio. Vediamo cosa succede:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Riesci a indovinare perché i punti si allineano in linee verticali in questo modo?

Abbiamo osservato la correlazione tra un concetto artificialmente creato come lo stipendio e la variabile osservata *altezza*. Vediamo anche se le due variabili osservate, come altezza e peso, sono correlate:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Sfortunatamente, non abbiamo ottenuto alcun risultato - solo alcuni strani valori `nan`. Ciò è dovuto al fatto che alcuni dei valori nella nostra serie non sono definiti, rappresentati come `nan`, il che causa che il risultato dell'operazione sia indefinito altrettanto. Osservando la matrice possiamo vedere che `Weight` è la colonna problematica, perché è stata calcolata l'auto-correlazione tra i valori di `Height`.

> Questo esempio mostra l'importanza della **preparazione** e **pulizia** dei dati. Senza dati adeguati non possiamo calcolare nulla.

Usiamo il metodo `fillna` per riempire i valori mancanti e calcoliamo la correlazione: 


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

C'è infatti una correlazione, ma non così forte come nel nostro esempio artificiale. Infatti, se guardiamo al grafico a dispersione di un valore in funzione dell'altro, la relazione sarebbe molto meno evidente:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Conclusione

In questo notebook abbiamo imparato come eseguire operazioni di base sui dati per calcolare funzioni statistiche. Ora sappiamo come utilizzare un solido apparato di matematica e statistica per dimostrare alcune ipotesi, e come calcolare intervalli di confidenza per variabili arbitrarie dato un campione di dati.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo a garantire l’accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa deve essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali fraintendimenti o interpretazioni errate derivanti dall’uso di questa traduzione.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
